In [38]:
import pandas as pd
import geopandas as gpd
import numpy as np
import time
import subprocess
from shapely.geometry import MultiPolygon
import pickle
# from all_config import *
import os
# import gem_tracker_maps.trackers.save_file_to_s3 as save_file_to_s3
import save_file_to_s3
from shapely import wkt


In [41]:
# FIND 8 missing

lastegtfile = '/Users/gem-tah/Downloads/europe_2025-02-25.geojson'
currentegtfile = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-05-02.geojson'

In [49]:
# Read the parquet files into GeoDataFrames
lastegt = gpd.read_file(lastegtfile)
currentegt = gpd.read_file(currentegtfile) # filter down to europe only

for df in [lastegt, currentegt]:
    print(df.columns)

no_geo = lastegt[lastegt['geometry'].is_empty]
no_geo2 = currentegt[currentegt['geometry'].is_empty]
no_geo_all = pd.concat([no_geo, no_geo2])
print(len(no_geo_all))

# Ensure both GeoDataFrames have a common column to compare (e.g., 'url')
common_column = 'url'

# Find rows in currentegt that are not in lastegt
missing_in_lastegt = currentegt[~currentegt[common_column].isin(lastegt[common_column])]

# Find rows in lastegt that are not in currentegt
missing_in_currentegt = lastegt[~lastegt[common_column].isin(currentegt[common_column])]

# Display the differences
print("Rows in currentegt but not in lastegt:")
print(missing_in_lastegt)

print("\nRows in lastegt but not in currentegt:")
print(missing_in_currentegt)


Index(['id', 'url', 'name', 'unit-name', 'status', 'areas', 'other-name',
       'fuel', 'owner', 'parent', 'start-year', 'capacity', 'location',
       'prefecture/district', 'state/province', 'region', 'tracker',
       'tracker-acro', 'fuel-filter', 'maturity', 'tracker-custom',
       'other-local', 'subnat', 'subnat2', 'region2', 'pci5', 'pci6', 'pid',
       'prod-gas', 'prod-year-gas', 'scaling-capacity', 'capacity-table',
       'units-of-m', 'status-legend', 'areas-subnat-sat-display',
       'tracker-display', 'pci-list', 'geometry'],
      dtype='object')
Index(['url', 'areas', 'name', 'unit-name', 'fuel', 'capacity', 'status',
       'start-year', 'owner', 'parent', 'location-accuracy', 'subnat',
       'region', 'pid', 'id', 'tracker-acro', 'fuel-filter', 'maturity',
       'tracker-custom', 'facilitytype', 'subnat2', 'region2', 'pci5', 'pci6',
       'Latitude', 'Longitude', 'Operator', 'prod-oil', 'prod-year-oil',
       'prod-gas', 'prod-year-gas', 'scaling-capacity', '

/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_62364/3821570098.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  no_geo_all = pd.concat([no_geo, no_geo2])


In [10]:
ggit_egt = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/latest/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02_DATA_TEAM_COPY.parquet'

ggit_gaspipes = 'https://publicgemdata.nyc3.cdn.digitaloceanspaces.com/latest/GEM-GGIT-Gas-Pipelines-2024-12_DATA_TEAM_COPY.parquet'


In [26]:
# Read the parquet files into GeoDataFrames
egt_gdf = pd.read_parquet(ggit_egt)
gaspipes_gdf = pd.read_parquet(ggit_gaspipes) # filter down to europe only

egt_gdf = egt_gdf[['Wiki', 'geometry']]
gaspipes_gdf = gaspipes_gdf[['Wiki', 'geometry']]

for df in [egt_gdf, gaspipes_gdf]:
    print(df.columns)
    
#     df['geometry'] = df['geometry'].apply(lambda geom: wkt.loads(geom) if geom else None)

#     gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Ensure both GeoDataFrames have a common column to compare (e.g., 'url')
common_column = 'Wiki'

# Find rows in egt_gdf that are not in gaspipes_gdf
missing_in_gaspipes = egt_gdf[~egt_gdf[common_column].isin(gaspipes_gdf[common_column])]

# Find rows in gaspipes_gdf that are not in egt_gdf
missing_in_egt = gaspipes_gdf[~gaspipes_gdf[common_column].isin(egt_gdf[common_column])]

# Display the differences
print("Rows in EGT but not in GasPipes:")
print(missing_in_gaspipes)

print("\nRows in GasPipes but not in EGT:")
print(missing_in_egt)

# check if the 5 missing from new egt are in the egt file

missing = [
    "https://www.gem.wiki/Leningrad-Vyborg-State_Border_Gas_Pipeline",
    "https://www.gem.wiki/Gryazovets-Vyborg_Gas_Pipeline",
    "https://www.gem.wiki/Novopskov-Aksai-Mozdok_Pipeline",
    "https://www.gem.wiki/Stavropol_Moscow_Pipeline",
    "https://www.gem.wiki/Azerbaijan-Georgia-Romania_Interconnector_(AGRI)"
]

missing_not_in_egt = [link for link in missing if link not in egt_gdf[common_column].values]

print("Links in 'missing' that are not in egt_gdf:")
print(missing_not_in_egt)


#     

# read the parquet files, see what pipelines are missing between egt and gaspipes

Index(['Wiki', 'geometry'], dtype='object')
Index(['Wiki', 'geometry'], dtype='object')
Rows in EGT but not in GasPipes:
                                                   Wiki  \
27    https://www.gem.wiki/Bosnia_and_Herzegovina%E2...   
30    https://www.gem.wiki/Ionian_Adriatic_Gas_Pipeline   
79       https://www.gem.wiki/H2ercules_Pipeline_System   
81       https://www.gem.wiki/H2ercules_Pipeline_System   
82       https://www.gem.wiki/H2ercules_Pipeline_System   
...                                                 ...   
1267       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1268       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1269       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1270       https://www.gem.wiki/Doing_Hydrogen_Pipeline   
1271   https://www.gem.wiki/Pomeranian_Hydrogen_Cluster   

                                               geometry  
27    MULTILINESTRING ((16.3718 43.573, 16.5838 43.5...  
30    LINESTRING (19.559838 40.726419, 19.542955 40...

In [35]:
# EU gas pipelines discrepancy
# previous 978
# current 966 
# Missing 12
# now missing 7 after patchingin 5 from last europe map file
old_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-02-25.geojson'
new_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-05-02.geojson'

# find which units are missing in new file

# Load the old and new files as GeoDataFrames
old_gdf = gpd.read_file(old_file)
new_gdf = gpd.read_file(new_file)

# Ensure the data contains a column that uniquely identifies each pipeline (e.g., 'Pipeline ID' or 'Name')
# Replace 'Pipeline ID' with the actual column name that uniquely identifies pipelines

# old_pipelines = old_gdf[old_gdf['tracker-display']=='gas pipeline']  # Adjust columns as needed
# new_pipelines = new_gdf[new_gdf['tracker-display']=='gas pipeline']  # Adjust columns as needed

old_gdf = old_gdf[~old_gdf['geometry'].is_empty]
new_gdf = new_gdf[~new_gdf['geometry'].is_empty]
# old_pipelines = old_pipelines[['url', 'name']]  # Adjust columns as needed
# new_pipelines = new_pipelines[['url', 'name']]  # Adjust columns as needed

# Find pipelines in the old file that are missing in the new file
missing_pipelines = old_gdf[~old_gdf['url'].isin(new_gdf['url'])]

# Display the missing pipelines
print(missing_pipelines)
# Save the missing pipelines to a GeoJSON file
missing_pipelines.to_file(f"missing_pipelines_egt_2025-05-02.geojson", driver="GeoJSON")

         id                                                url  \
1262  P5590  https://www.gem.wiki/Leningrad-Vyborg-State_Bo...   
1312  P0746  https://www.gem.wiki/Gryazovets-Vyborg_Gas_Pip...   
1417  P1471  https://www.gem.wiki/Novopskov-Aksai-Mozdok_Pi...   
1489  P0793     https://www.gem.wiki/Stavropol_Moscow_Pipeline   
1503  P0735  https://www.gem.wiki/Azerbaijan-Georgia-Romani...   

                                                   name unit-name     status  \
1262         Leningrad-Vyborg-State Border Gas Pipeline        II  operating   
1312                     Gryazovets-Vyborg Gas Pipeline         I  operating   
1417                    Novopskov-Aksai-Mozdok Pipeline            operating   
1489                      Stavropol-Moscow Gas Pipeline              retired   
1503  Azerbaijan-Georgia-Romania Interconnector Gas ...             proposed   

                     areas other-name fuel  \
1262      Estonia; Russia;             Gas   
1312               Russia;    

In [50]:
# fix current egt data
missing = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/missing_pipelines_egt_2025-05-02.geojson'
current = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/europe_2025-05-05.geojson'
path_eu = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/europe/compilation_output/'

missing_gdf = gpd.read_file(missing)
current_gdf = gpd.read_file(current)

# Concatenate the GeoDataFrames
fixed = gpd.GeoDataFrame(pd.concat([missing_gdf, current_gdf], ignore_index=True))


# Save the resulting GeoDataFrame if needed
fixed.to_file(f"{path_eu}europe_i5_2025-05-05.geojson", driver="GeoJSON")



In [37]:
# Extract all unique countries from the 'areas' column, split by ';', and alphabetize them
countries = sorted(set(country.strip() for area in fixed['areas'] for country in area.split(';') if country.strip()))
countries

['Albania',
 'Algeria',
 'Austria',
 'Azerbaijan',
 'Belarus',
 'Belgium',
 'Benin',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Egypt',
 'Estonia',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Guinea',
 'Guinea-Bissau',
 'Hungary',
 'Iran',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Jordan',
 'Kazakhstan',
 'Kosovo',
 'Latvia',
 'Liberia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Mauritania',
 'Moldova',
 'Montenegro',
 'Morocco',
 'Netherlands',
 'Nigeria',
 'North Macedonia',
 'Norway',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russia',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'Sierra Leone',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Syria',
 'The Gambia',
 'Togo',
 'Tunisia',
 'Türkiye',
 'Ukraine',
 'United Kingdom']

In [2]:
# Define the column headers as lists
plants_columns = [
    "Wiki URL", "Country/Area", "Plant name", "Plant Name in Local Language / Script", "Other Name(s)", 
    "Unit name", "Fuel", "Capacity (MW)", "Status", "Turbine/Engine Technology", 
    "Equipment Manufacturer/Model", "CHP", "Hydrogen capable?", "CCS attachment?", 
    "Conversion/replacement?", "Conversion from/replacement of (fuel)", 
    "Conversion from/replacement of (GEM unit ID)", "Conversion to (fuel)", 
    "Conversion to (GEM unit ID)", "Start year", "Retired year", "Planned retire", 
    "Operator(s)", "Owner(s)", "Parent(s)", "Latitude", "Longitude", "Location accuracy", 
    "City", "Local area (taluk, county)", "Major area (prefecture, district)", 
    "State/Province", "Subregion", "Region", "Other IDs (location)", "Other IDs (unit)", 
    "Captive industry use", "Captive industry type", "Captive non-industry use", 
    "GEM location ID", "GEM unit ID"
]

plants_hy_columns = [
    "In Oil & Gas Plants tab?", "Wiki URL", "Country/Area", "Plant name", 
    "Plant Name in Local Language / Script", "Other Name(s)", "Unit name", "Fuel", 
    "Capacity (MW)", "Status", "Turbine/Engine Technology", "Equipment Manufacturer/Model", 
    "CHP", "Hydrogen capable?", "CCS attachment?", "Conversion/replacement?", 
    "Conversion from/replacement of (fuel)", "Conversion from/replacement of (GEM unit ID)", 
    "Conversion to (fuel)", "Conversion to (GEM unit ID)", "Start year", "Retired year", 
    "Planned retire", "Operator(s)", "Owner(s)", "Parent(s)", "Latitude", "Longitude", 
    "Location accuracy", "City", "Local area (taluk, county)", "Major area (prefecture, district)", 
    "State/Province", "Subregion", "Region", "Other IDs (location)", "Other IDs (unit)", 
    "Captive industry use", "Captive industry type", "Captive non-industry use", 
    "GEM location ID", "GEM unit ID", "H2 usage proposed %", "Color of H2 proposed?", 
    "H2 ready turbine (%)?", "MOU for H2 supply?", "Contract for H2 supply?", 
    "Financing for supply of H2?", "Co-located with electrolyzer/H2 production facility?", 
    "What % of H2 blending currently?"
]

# Find the differences between the two lists
not_in_plants_hy = list(set(plants_columns) - set(plants_hy_columns))
not_in_plants = list(set(plants_hy_columns) - set(plants_columns))

# Combine the results into a single list
non_overlapping_columns = {
    "Not in plants_hy_columns": not_in_plants_hy,
    "Not in plants_columns": not_in_plants
}

non_overlapping_columns



{'Not in plants_hy_columns': [],
 'Not in plants_columns': ['MOU for H2 supply?',
  'In Oil & Gas Plants tab?',
  'What % of H2 blending currently?',
  'Contract for H2 supply?',
  'H2 usage proposed %',
  'Financing for supply of H2?',
  'Co-located with electrolyzer/H2 production facility?',
  'Color of H2 proposed?',
  'H2 ready turbine (%)?']}

In [14]:
# file = 'local_pkl/about_df_dict_by_map_2025-02-07.pkl'
filegoit =  'GOIT/compilation_output/goit_2025-04-08.geojson'
# file_goget_json = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem_tracker_maps/trackers/goget/compilation_output/Oil & Gas Extraction-map-file-2025-02-20.csv'

In [ ]:
# https://drive.google.com/file/d/1wy53Z56YeIad4Tod46tGcOmduxBPLZ7r/view?usp=drive_link can we access via this link?

ggit_lng_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson'
ggit_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-Gas-Pipelines-2024-12 DATA TEAM COPY.geojson' #'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-Gas-Pipelines-2023-12 copy.geojson'

ggit_lng_eu_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-EGT-Terminals-2025-02 DATA TEAM COPY.geojson'
ggit_eu_geojson = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02 DATA TEAM COPY.geojson'

list_of_geojsons = [ggit_lng_geojson, ggit_geojson, ggit_lng_eu_geojson, ggit_eu_geojson]

for filepath in list_of_geojsons:
    gdf = gpd.read_file(filepath)
    print(gdf.shape)
    # newfilepath = '/'.join(filepath.split('/')[:-1])
    newfilepath = filepath.replace('.geojson', '.parquet').replace(' ', '_')
    print(f'{newfilepath}')
    gdf.to_parquet(path = f'{newfilepath}')
    # save_file_to_s3.saves3(f'{filepath}') # also save original
    save_file_to_s3.saves3(newfilepath)

(1295, 59)
/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson


CalledProcessError: Command 'export BUCKETEER_BUCKET_NAME=publicgemdata && aws s3 cp /Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson s3://$BUCKETEER_BUCKET_NAME/latest/ --endpoint-url https://nyc3.digitaloceanspaces.com --acl public-read' returned non-zero exit status 252.

In [ ]:
# save most recent download to s3


In [ ]:
# list_of_links = ['https://docs.google.com/spreadsheets/d/1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0/', 'https://drive.google.com/file/d/1X6tpYmgDKUWZzSYGlz0LcCeE-DqGeQcp/view?usp=drive_link', 
#                  'https://docs.google.com/spreadsheets/d/1aZiKzbHfFXLSEFCeKwstvL1-DV98ErXDhyCx00abRgQ/', 'https://docs.google.com/spreadsheets/d/1dqawuLhgFzYI9GWRU9bwsDY1SQHjzxifZb7u1T8OXqA/',
#                  'https://docs.google.com/spreadsheets/d/1zUyj7t4FO0M9fc82wpG93fZmisGar0Syq1l5ZGyyxEM/', 'https://docs.google.com/spreadsheets/d/1TxoQ17GUDvRbQ82r90eexbRNHfPfi-50wm9lBih2eYc/',
#                  'https://docs.google.com/spreadsheets/d/1hB2NuXMMqCJD94muoe6-A9BCi4v-7hquScej44_QWas/', 'https://docs.google.com/spreadsheets/d/1crtLnGzOS6wLYweIb84gLQpMV4A49qD5fty6WBTdLt0/',
#                  'https://docs.google.com/spreadsheets/d/1jUbsItBq_CVXy0dUxCP7APHfstDL85BaykCXYURagX4/', 'https://docs.google.com/spreadsheets/d/1bUTPRTmIqu7URVECm66NGr_DF7v32UUOzBaBf1dNawk/',
#                  'https://docs.google.com/spreadsheets/d/1x-IxdiqMJ_v-M_45ASFdg55Gl3Hsx3vb-7wjt8qCq5k/', 'https://docs.google.com/spreadsheets/d/1cy0A7k0GoB2b_b2wesObcAvts0GbXa7go7FVE_bHpBc/',
#                  'https://docs.google.com/spreadsheets/d/1XxAvcobKZ9XRKvHaSHDb4JbOstqhvrVWkU5Wu8JKhgE/', 'https://docs.google.com/spreadsheets/d/1gOOwjN551r6TllpBpfxtwNqF1yfbLbXdCFwACvKYeI4/',
#                  'https://docs.google.com/spreadsheets/d/10Clsb8auutXy4iBm6c7HywZjSbNKUiho9NHzZmC2-Bs/', 'https://docs.google.com/spreadsheets/d/1cqLe0xOx7FLpfaGZhtVnV-bhuKgaczHPtXzCRIgy_RQ/',
#                  'https://docs.google.com/spreadsheets/d/1zCcO9LwVmiQZBOGqlwKGYgZ5lAnBKJ91hjwWcbUw5nQ/', 'https://docs.google.com/spreadsheets/d/1zCcO9LwVmiQZBOGqlwKGYgZ5lAnBKJ91hjwWcbUw5nQ/',
#                  'https://docs.google.com/spreadsheets/d/16MoNKcz68GVwv6ztZ6p0vSNlL6EmPd633DZ7V0xG_Ho/']


# for link in list_of_links:
    
# save_file_to_s3.googlelinktoparquet(link)

Most recent file found: /Users/gem-tah/Downloads/Global Coal Project Finance Tracker_April 2025_Global Energy Monitor DATA TEAM COPY.xlsx
most_recent_file_name: Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY
File saved as Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY_1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0_2025-04-14.parquet
upload: ./Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY_1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0_2025-04-14.parquet to s3://publicgemdata/latest/Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY_1wI11cMqhqZXTK7MVD2semkZIqlF4DSpfsNTaSPBWWO0_2025-04-14.parquet

Most recent file found: /Users/gem-tah/Downloads/Global Coal Project Finance Tracker_April 2025_Global Energy Monitor DATA TEAM COPY.xlsx
most_recent_file_name: Global_Coal_Project_Finance_Tracker_April_2025_Global_Energy_Monitor_DATA_TEAM_COPY
File saved a

KeyboardInterrupt: 

In [15]:
df = gpd.read_file(filegoit)


In [ ]:
# df.to_parquet(f"goit-map-2025-03.parquet", index=False)  # partition_cols=["country/area"],
# file = 'goit-map-2025-03.parquet'
file = filegoit

do_command_s3 = (
            f'export BUCKETEER_BUCKET_NAME=publicgemdata && '
            f'aws s3 cp {file} s3://$BUCKETEER_BUCKET_NAME/latest/ '
            f'--endpoint-url https://nyc3.digitaloceanspaces.com --acl public-read')

        # Execute the terminal command to pull down file from digital ocean
process = subprocess.run(do_command_s3, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print(process.stdout.decode('utf-8'))
if process.stderr:
    print(process.stderr.decode('utf-8'))


upload: GOIT/compilation_output/goit_2025-04-08.geojson to s3://publicgemdata/latest/goit_2025-04-08.geojson



In [11]:
set(df['status-legend'].to_list())

{'cancelled',
 'construction-plus',
 'mothballed-plus',
 'operating',
 'proposed-plus',
 'retired-plus',
 'shelved'}

In [ ]:
# df = pickle.load(file)
# df = gpd.read_file(file_goget_json)

In [ ]:

# country_list = set(df['country-list'].to_list())
# print(country_list)

countries = set(df['country/area'].apply(lambda x: x.split('-')).apply(lambda x: '; '.join(x)).to_list())
print(sorted(countries))

# missing = country_list - countries


# print(missing)

In [ ]:
current_countries = {
  "Africa": [
    "Algeria",
    "Angola",
    "Angola-Republic of the Congo",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "Côte d'Ivoire",
    "Djibouti",
    "DR Congo",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Republic of the Congo",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Senegal-Mauritania",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ],
  "Americas": [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bermuda",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Canada",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Greenland",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Puerto Rico",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Pierre and Miquelon",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "United States",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)",
    "Virgin Islands (U.S.)"
  ],
  "Asia": [
    "Afghanistan",
    "Armenia",
    "Azerbaijan",
    "Azerbaijan-Turkmenistan",
    "Bahrain",
    "Bangladesh",
    "Bhutan",
    "Brunei",
    "Cambodia",
    "China",
    "China-Japan",
    "Cyprus",
    "Georgia",
    "Hong Kong",
    "India",
    "Indonesia",
    "Iran",
    "Iran-Iraq",
    "Iraq",
    "Israel",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kuwait",
    "Kuwait-Saudi Arabia",
    "Kuwait-Saudi Arabia-Iran",
    "Kyrgyzstan",
    "Laos",
    "Lebanon",
    "Macao",
    "Malaysia",
    "Maldives",
    "Mongolia",
    "Myanmar",
    "Nepal",
    "North Korea",
    "Oman",
    "Pakistan",
    "Palestine",
    "Philippines",
    "Qatar",
    "Russia-Kazakhstan",
    "Saudi Arabia",
    "Saudi Arabia-Bahrain",
    "Saudi Arabia-Iran",
    "Singapore",
    "South China Sea",
    "South Korea",
    "Sri Lanka",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Thailand",
    "Thailand-Malaysia",
    "Timor-Leste",
    "Türkiye",
    "Turkmenistan",
    "United Arab Emirates",
    "United Arab Emirates-Iran",
    "Uzbekistan",
    "Vietnam",
    "Vietnam-Malaysia",
    "Yemen"
  ],
  "Europe": [
    "Åland Islands",
    "Albania",
    "Andorra",
    "Austria",
    "Belarus",
    "Belgium",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Faroe Islands",
    "Finland",
    "France",
    "Germany",
    "Gibraltar",
    "Greece",
    "Guernsey",
    "Holy See",
    "Hungary",
    "Iceland",
    "Ireland",
    "Isle of Man",
    "Italy",
    "Jersey",
    "Kosovo",
    "Latvia",
    "Liechtenstein",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Moldova",
    "Monaco",
    "Montenegro",
    "Netherlands",
    "North Macedonia",
    "Norway",
    "Poland",
    "Portugal",
    "Romania",
    "Russia",
    "Russia-Kazakhstan",
    "San Marino",
    "Serbia",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Svalbard and Jan Mayen",
    "Sweden",
    "Switzerland",
    "Ukraine",
    "United Kingdom"
  ],
  "Oceania": [
    "American Samoa",
    "Australia",
    "Christmas Island",
    "Cocos (Keeling) Islands",
    "Cook Islands",
    "Fiji",
    "French Polynesia",
    "Guam",
    "Heard Island and McDonald Islands",
    "Kiribati",
    "Marshall Islands",
    "Micronesia",
    "Nauru",
    "New Caledonia",
    "New Zealand",
    "Niue",
    "Norfolk Island",
    "Northern Mariana Islands",
    "Palau",
    "Papua New Guinea",
    "Pitcairn",
    "Samoa",
    "Solomon Islands",
    "Timor Gap",
    "Tokelau",
    "Tonga",
    "Tuvalu",
    "United States Minor Outlying Islands",
    "Vanuatu",
    "Venezuela-Trinidad and Tobago",
    "Wallis and Futuna"
  ]
}


current_countries_list = []

for v in current_countries.values():
    current_countries_list.extend(v)

print(current_countries_list)

print(current_countries_list)

In [ ]:
countries_list = list(countries)
countries_list = [c.replace('; ', '-') for c in countries_list]

missing_countries_areas = set(countries_list) - set(current_countries_list)

In [ ]:
missing_countries_areas

In [ ]:
# add missing_countries_areas to current_countries 


compare all regional outputs and source data

In [ ]:
folder_path_pkl = f'{tracker_folder_path}local_pkl'


for folder in ['LATAM', 'africa-energy', 'asia']:
    
    file_path = f'{folder}/config.js'
    if folder == 'africa-energy':
        folder = 'africa'
        
    with open(file_path, 'r') as file:
        jscontent = file.read()
        # print(type(jscontent)) # string
        # print(jscontent)
        # make df from json file 
        if 'geojson:' in jscontent:
            second_part = jscontent.split('json: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]
            folder_full = tracker_folder_path + str(folder)


            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')
            current_file_line = str(current_file_line)
            print(current_file_line)
            current_file_line_csv = current_file_line.replace('geojson', 'csv')

            full_path = os.path.join(folder_full,current_file_line)

            # df = pd.read_json(full_path)
            # change to csv version that is / should be saved next to it b/c easier to read
            full_path_csv = os.path.join(folder_full,current_file_line_csv)
            try:
                df = pd.read_csv(full_path_csv)
            except FileNotFoundError:
                current_file_line_xlsx = current_file_line.replace('json', 'xlsx')
                full_path_xlsx = os.path.join(folder_full,current_file_line_xlsx)
                print(full_path_xlsx)
                try: 
                    df = pd.read_excel(full_path_xlsx)
                    
                except FileNotFoundError:
                    print(f'Check the folder {folder_full} seemingly no alternatives to json tried csv and xlsx')
            finally:
                # print(df.head())
                print(f'Length df: {len(df)}')

                
        
        elif 'json' in jscontent:
            second_part = jscontent.split('json: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]

            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')

            folder_full = tracker_folder_path + str(folder)

            current_file_line = str(current_file_line)
            print(current_file_line)
            current_file_line_csv = current_file_line.replace('geojson', 'csv')

            full_path = os.path.join(folder_full,current_file_line)

            # df = pd.read_json(full_path)
            # change to csv version that is / should be saved next to it b/c easier to read
            full_path_csv = os.path.join(folder_full,current_file_line_csv)
            try:
                df = pd.read_csv(full_path_csv)
            except FileNotFoundError:
                current_file_line_xlsx = current_file_line.replace('geojson', 'xlsx')
                full_path_xlsx = os.path.join(folder_full,current_file_line_xlsx)
                print(full_path_xlsx)
                try: 
                    df = pd.read_excel(full_path_xlsx)
                except FileNotFoundError:
                    print(f'Check the folder {folder_full} seemingly no alternatives to json tried csv and xlsx')
            finally:
                # print(df.head())
                # print(len(df))
                print(f'Length df: {len(df)}')

            
        elif 'csv:' in jscontent:
            second_part = jscontent.split('csv: ')[1]
            current_file_line = second_part.split(",")[0]
            print(f'For {folder} map: {current_file_line}')
            current_file_line = current_file_line.replace("'","")
            date_pkl = current_file_line.split(f'{folder.lower()}_')[1].split('.')[0]

            file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'
            full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
            df_pkl = pd.read_pickle(full_path_pkl)
            print(f'Length of source after being filtered: {len(df_pkl)}')


            folder_full = tracker_folder_path + str(folder)
            full_path = os.path.join(folder_full,current_file_line)
            df = pd.read_csv(full_path)
            print(f'Length df: len(df)')

            
        else:
            print(jscontent)

        print(date_pkl)
        # filter for tracker to update 
        print(folder)
        print(len(df))
        df_filt = df[df["tracker-acro"]=='GOGET']
        print(len(df_filt))

            
    

totals:
AGT: Map - 474, table - 479 (potentially Thailand-Malaysia, China-Japan, Vietnam-Malaysia missing)
LATAM - map - 1195, table - 1200 (missing- Venezuela-Trinidad and Tobago?)
AET:Map - 665, table 667 (Senegal-Mauritania, Angola-Republic of the Congo)

In [ ]:
# pull in source
# filter for region and fuel
folder_path_pkl = f'{tracker_folder_path}local_pkl'
file_map_tracker = f'{folder}_{trackers_to_update[0]}_df_{date_pkl}.pkl'

full_path_pkl = os.path.join(folder_path_pkl,file_map_tracker)
df_pkl = pd.read_pickle(full_path_pkl)
f'Length of source after being filtered: {len(df_pkl)}'


Check the summary colab to see how he filters region and fuel, must be that. 
And pull in europe to this slew of functions so have those pkl files too

From scotts summary code 
Regional Tracker Summary Tables
https://colab.research.google.com/drive/1c1Y7MHV6f8lEAeb8l0nw1B_d1ETmwVRb#scrollTo=e4yXRRIkch6v 

In [ ]:
# AET covers "The Africa Energy Tracker covers the subregions of Northern Africa and Sub-Saharan Africa" https://globalenergymonitor.org/projects/africa-energy-tracker/methodology/
# Define African Sites
african_countries = [
    "Algeria", "Angola", "Angola-Republic of the Congo", "Benin", "Botswana",
    "British Indian Ocean Territory", "Burkina Faso", "Burundi", "Cabo Verde",
    "Cameroon", "Central African Republic", "Chad", "Comoros", "Côte d'Ivoire",
    "Djibouti", "DR Congo", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini",
    "Ethiopia", "French Southern Territories", "Gabon", "Ghana", "Guinea",
    "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar",
    "Malawi", "Mali", "Mauritania", "Mauritius", "Mayotte", "Morocco",
    "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo",
    "Réunion", "Rwanda", "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe", "Senegal", "Senegal-Mauritania", "Seychelles",
    "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan",
    "Tanzania", "The Gambia", "Togo", "Tunisia", "Uganda", "Western Sahara",
    "Zambia", "Zimbabwe"
]


# Define Asian Countries https://globalenergymonitor.org/projects/asia-gas-tracker/methodology/
asia_countries = [
    # South Asia
    "Afghanistan", "Bangladesh", "Bhutan", "India", "Iran",
    "Maldives", "Nepal", "Pakistan", "Sri Lanka",

    # Southeast Asia
    "Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia",
    "Myanmar", "Philippines", "Singapore", "Thailand",
    "Timor-Leste", "Vietnam", "Thailand-Malaysia",  "Vietnam-Malaysia",

    # East Asia
    "China", "China-Japan", "Hong Kong", "Japan", "Macao",
    "Mongolia", "North Korea", "South Korea", "Taiwan",

    # Multinational or Maritime Areas
    "South China Sea"
]

# Define European Countries and Territories https://globalenergymonitor.org/projects/europe-gas-tracker/methodology/
europe_countries = [
    # Northern Europe
    "Åland Islands", "Denmark", "Estonia", "Faroe Islands", "Finland",
    "Iceland", "Ireland", "Isle of Man", "Latvia", "Lithuania",
    "Norway", "Sweden", "United Kingdom", "Guernsey", "Jersey", "Svalbard and Jan Mayen",

    # Western Europe
    "Austria", "Belgium", "France", "Germany", "Liechtenstein",
    "Luxembourg", "Monaco", "Netherlands", "Switzerland",

    # Eastern Europe
    "Belarus", "Czech Republic", "Hungary", "Moldova", "Poland",
    "Romania", "Slovakia", "Ukraine",

    # Southern Europe
    "Albania", "Andorra", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Cyprus", "Gibraltar", "Greece", "Italy", "Kosovo", "Malta",
    "Montenegro", "North Macedonia", "Portugal", "San Marino",
    "Serbia", "Slovenia", "Spain", "Türkiye",

    # Special Cases
    "Israel"  # list in the methodology
]

# Define Latin American and Caribbean Countries and Territories - see "Data request for the new year"
latin_america_caribbean = [
    # Caribbean
    "Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados",
    "Belize", "Cayman Islands", "Cuba", "Curaçao", "Dominica",
    "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica",
    "Martinique", "Montserrat",  "Saint Barthélemy",
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Martin (French part)",
    "Saint Vincent and the Grenadines", "Sint Maarten (Dutch part)",
    "Trinidad and Tobago", "Turks and Caicos Islands", "Virgin Islands (British)",
   # "Virgin Islands (U.S.)", "Puerto Rico", -  gregor excludes

    # Central America
    "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico",
    "Nicaragua", "Panama",

    # South America
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador",
    "French Guiana", "Guyana", "Paraguay", "Peru", "Suriname",
    "Uruguay", "Venezuela",

    # Special Cases
    "Bonaire, Sint Eustatius, and Saba", "Bouvet Island",
    "Falkland Islands", "South Georgia and the South Sandwich Islands",
    "Venezuela-Trinidad and Tobago"
]

In [ ]:
# map countries 


africa_countries_map = [
    "Algeria",
    "Angola",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "DR Congo",
    "Republic of the Congo",
    "Côte d'Ivoire",
    "Djibouti",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ]


asia_countries_map = [ 
        "China",
        "Hong Kong",
        "Japan",
        "Macao",
        "Mongolia",
        "North Korea",
        "South Korea",
        "Taiwan",
        "Brunei",
        "Cambodia",
        "Indonesia",
        "Laos",
        "Malaysia",
        "Myanmar",
        "Philippines",
        "Singapore",
        "Thailand",
        "Timor-Leste",
        "Vietnam",
        "Afghanistan",
        "Iran",
        "Bangladesh",
        "Bhutan",
        "India",
        "Maldives",
        "Nepal",
        "Pakistan",
        "Sri Lanka"]

# european_union_countries = [
#     'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
#     'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 
#     'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 
#     'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 
#     'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 
#     'Spain', 'Sweden',
# ]
# other_europe_countries = [
#     'Albania', 'Andorra', 'Belarus', 'Bosnia and Herzegovina', 'Holy See', 'Iceland',
#     'Liechtenstein', 'Moldova', 'Monaco', 'Montenegro', 'North Macedonia', 
#     'Norway', 'San Marino', 'Serbia', 'Switzerland', 'Türkiye', 'Ukraine', 
#     'United Kingdom', 'Israel',
# ]

europe_countries_map = [
    'Åland Islands', 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 
    'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czech Republic', 
    'Denmark', 'Estonia', 'Faroe Islands', 'Finland', 'France', 'Germany', 
    'Gibraltar', 'Greece', 'Guernsey', 'Holy See', 'Hungary', 'Iceland', 
    'Ireland', 'Isle of Man', 'Italy', 'Jersey', 'Kosovo', 'Latvia', 
    'Liechtenstein', 'Lithuania', 'Luxembourg', 'North Macedonia', 'Malta', 
    'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 
    'Portugal', 'Romania', 'Israel', 'San Marino', 'Serbia', 'Slovakia', 
    'Slovenia', 'Spain', 'Svalbard and Jan Mayen', 'Sweden', 'Switzerland', 
    'Ukraine', 'United Kingdom', 'Cyprus', 'Türkiye'
]



latam_countries_map = [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)"]

In [ ]:
#compare differenc with subtract a set

missing_eu = set(europe_countries) - set(europe_countries_map) 
# missing_eu = set(europe_countries_map) - set(europe_countries) 

print(missing_eu)

missing_asia = set(asia_countries) - set(asia_countries_map)
# missing_asia = set(asia_countries_map) - set(asia_countries)

print(missing_asia)

missing_africa = set(african_countries) - set(africa_countries_map)
# missing_africa = set(africa_countries_map) - set(african_countries)

print(missing_africa)

missing_latam = set(latin_america_caribbean) - set(latam_countries_map)
# missing_latam = set(latam_countries_map) - set(latin_america_caribbean)

print(missing_latam)



In [ ]:
# map 949 goget eu
# scott 964.0 goget eu

GOGET architecture exploring

In [76]:
# 7k projects goget
goget_file = '/Users/gem-tah/Downloads/GOGET oil & gas extraction sites (main data set) - Main data.csv'

goget_df = pd.read_csv(goget_file)
print(set(goget_df['Project or complex'].to_list()))

# print(goget_df['Unit type'])
print(goget_df.columns)



{nan, '11416/Cold Lake', '10587DD/Athabasca', 'Carabobo-3', 'La Rona', 'Upper Wolfcamp East', 'Doe', '12689A/Athabasca', '8002F/Cold Lake', 'Girsan-Chulkuvar', 'Diatomite', '12229A/Cold Lake', '9349E/Cold Lake', 'Parkland', 'Penn Upper', '9637C/Cold Lake', 'Gannet', '12680A/Athabasca', 'Other Areas', 'Eastern Fields Oil and Gas Complex', 'MLNG Tiga', 'Brazion', 'Kenli 6-1', '10616/Peace River Area 2', '12316/Athabasca', '12795/Cold Lake', 'Xijiang 30-2', 'Sapphire', 'Gina Krog', 'Brush Creek', 'Peace River Area 2', 'Stevens', '13009/Peace River Area 2', '11480B/Athabasca', 'Maari', 'Wilcox 10400', '11256I/Cold Lake', 'Gela', 'Jedney', 'Surgil', 'North East Bab', 'Carneros', 'Coatzacoalcos-Marino; Area 1', 'Devonian', 'Buzzard Creek', 'Kurdistan', '11320F/Peace River Area 2', 'Pettit, Lower Gas', 'Banyu Urip', 'Jujo-Tecominoacán', 'Cv', 'Cold Lake Area - Scheme 8908T Oil Sands', 'Narwhal', 'Julimar-Brunello', 'Liuhua 16-2', 'Serrette', '11729A/Cold Lake', '10070J/Peace River Area 1', '1

In [77]:
projects = goget_df[goget_df['Unit type']=='project']
not_projects = goget_df[goget_df['Unit type']!='project']
print(len(projects))
print(len(not_projects))

not_in_project = goget_df[goget_df['Project or complex'].isna()]
print(len(not_in_project))

print(len(goget_df))

# are those that complexes that have no data but have fields associated with them get published in the official data download

# find the rows that are connected to a project,

         
# how many units are in a project 

626
14784
11094
15410


In [78]:
# we want a list of all the goget units that do not agree with its project
#  Unit type = field, project, complex 
# Project or complex = name of corresponding project 

In [86]:
hier_top = ['project', 'complex'] # block
hier_middle = ['field', 'area', 'asset', 'pool', 'phase']
hier_middle_new = ['field', 'area', 'asset', 'pool', 'phase', 'sub-basin', 'concession', 'basin']
# hier_low = [''] # 'sub-basin, concession, basin

cols_hier_top = ['Owner 1p', 'Owner 1f', 'Owner 1 %p', 'Owner 1 %f', 'Owner 2p', 'Owner 2f', 'Owner 2 %p', 'Owner 2 %f', 'Owner 3p', 'Owner 3f', 'Owner 3 %p', 'Owner 3 %f', 'Owner 4p', 'Owner 4f', 'Owner 4 %p', 'Owner 4 %f', 'Owner 5p', 'Owner 5f', 'Owner 5 %p', 'Owner 5 %f', 'Owner % sump', 'Owner % sumf'] # FID, start up, owner, onshore/offshore
cols_hier_middle = ['Latitudep', 'Latitudef', 'Longitudep', 'Longitudef', 'Location accuracyp', 'Location accuracyf', 'Operatorp', 'Operatorf'] # discover, operator, lat, long, basin, formation, block
cols_agree_needed = ['Country/Areap', 'Country/Areaf', 'Subnational unit (province, state)p', 'Subnational unit (province, state)f', 'Statusp', 'Statusf']
basic_cols = ['project ID (inferred)','project name (inferred)','Unit IDp','Unit IDf', 'Unit Namep','Unit Namef', 'Project or complexp', 'Project or complexf']

set(goget_df['Unit type'].to_list())
questions = '1.)what to do with 480 that arent clearly top or mid from that schema?'

In [91]:
# field project groupings for existing hierarchy data reconciliation
# a field is in a grouping if its Unit type = field and its Project or complex is not blank
# a project is in a grouping if its Unit type = project or complex and in a pivot table it has fields
# find situations where field and project do not correspond, match on name 

# groupings = [] # tuples of project first, then list of fields
field_gr = goget_df.copy()
print(len(field_gr))
# field_gr = field_gr[field_gr['Unit type'].isin(hier_middle)]
field_gr = field_gr[field_gr['Unit type'].isin(hier_middle_new)]

field_gr = field_gr[field_gr['Project or complex'].notna()] # 14177 without this filter 3540 with it
field_gr['Project or complex'] = field_gr['Project or complex'].apply(lambda x:x.strip())
print(len(field_gr))
field_proj_names = field_gr['Project or complex'].to_list()
print(field_proj_names)

proj_gr = goget_df.copy()
print(len(proj_gr))
proj_gr = proj_gr[proj_gr['Unit type'].isin(hier_top)]
proj_gr['Unit Name'] = proj_gr['Unit Name'].apply(lambda x:x.strip())
print(len(proj_gr))
proj_names = proj_gr['Unit Name'].to_list()
print(proj_names)

no_overlap = set(field_proj_names) - set(proj_names) 
print(no_overlap)
print(len(goget_df)-len(proj_gr)-len(field_gr)) # 480 neither field or project 


15410
3543
['Matzen', 'Parque das Conchas', 'Iara, Atapu', 'Parque das Conchas', 'Iara, Atapu', 'Bacalhau', 'Bacalhau', 'Parque das Baleias', 'Iara', 'Búzios', 'Parque das Baleias', 'Parque das Baleias', 'Parque dos Gaviões', 'Parque dos Gaviões', 'Parque das Baleias', 'Lapa', 'Mero/Libra', 'Iara, Atapu', 'Parque das Conchas', 'Parque das Baleias', 'Sepia', 'Sepia', 'Iara', 'Iara', 'Aphrodite', 'Calypso', 'Glaucus', 'Lacq', 'Lacq', 'Düste', 'Düste', 'Emlichheim', 'Emlichheim', 'Kalle', 'Kalle', 'Ratzel', 'Ratzel', 'Ratzel', 'Rehden', 'Rehden', 'Rehden', 'Ringe', 'Ringe', 'Rütenbrock', 'Rütenbrock', 'Voelkersen', 'Wielen', 'Wielen', 'Wietingsmoor', 'Wietingsmoor', 'Algyo', 'Kinsale', 'Corrib', 'Inishkea', 'Kinsale', 'Kinsale', 'Kinsale', 'Karish-Tanin', 'Karish-Tanin', 'Leviathan', 'Tamar', 'Karish-Tanin', 'Val d’Agri', 'Val d’Agri', 'Val d’Agri', 'Gateway to the Ems', 'Grieg / Aasen', 'Valhall', 'Johan Castberg', 'Johan Sverdrup', 'B 8', 'BMB', 'Brońsko', 'Przemyśl', 'Midia Gas Develop

In [92]:
# merge project rows with field rows 
print(cols_agree_needed + basic_cols)
groupings = proj_gr.merge(field_gr, left_on='Unit Name', right_on='Project or complex', how='inner', suffixes=('p', 'f'))
# rename Unit IDp to be project id for easier pivoting
# groupings.rename(columns={'Unit IDp': 'project ID (inferred)', 'Unit Namep': 'project name (inferred)'}, inplace=True)
groupings['project ID (inferred)'] = groupings['Unit IDp']
groupings['project name (inferred)'] = groupings['Unit Namep']
# len(groupings)
# print(groupings)


cols_rec_check = groupings[cols_agree_needed + basic_cols]
cols_rec_check_proj = groupings[cols_hier_top + basic_cols]
cols_rec_check_field = groupings[cols_hier_middle + basic_cols]

all_groupings = groupings[basic_cols + cols_agree_needed + cols_hier_top + cols_hier_middle]
# all_groupings.to_csv(f'all_groupings_goget 2025-03-10.csv')


['Country/Areap', 'Country/Areaf', 'Subnational unit (province, state)p', 'Subnational unit (province, state)f', 'Statusp', 'Statusf', 'project ID (inferred)', 'project name (inferred)', 'Unit IDp', 'Unit IDf', 'Unit Namep', 'Unit Namef', 'Project or complexp', 'Project or complexf']


In [95]:

# groupings
# cols_rec_check
# cols_rec_check_proj
# cols_rec_check_field
# pivot 
# group on project ID and find inconsistencies
# pivot_cols = basic_cols.pop(2)
# del basic_cols[2] # project ID (inferred)
# pivot_cols = [basic_cols.pop(basic_cols.index('project ID (inferred)'))]
# pivot_cols = cols_agree_needed + cols_hier_top + cols_hier_middle
# all_groupings = all_groupings.groupby(['project ID (inferred)','project name (inferred)'])
# all_groupings = all_groupings.pivot(index='project ID (inferred)', columns = pivot_cols)
# print(all_groupings)

# a file showing side by side cols that do not match between project and field, but so that each field is lined up along project
# cols_hier_top = ['Owner 1', 'Owner 1 %', 'Owner 2', 'Owner 2 %', 'Owner 3', 'Owner 3 %', 'Owner 4', 'Owner 4 %', 'Owner 5', 'Owner 5 %', 'Owner % sum'] # FID, start up, owner, onshore/offshore
# cols_hier_middle = ['Latitude', 'Longitude', 'Location accuracy', 'Operator'] # discover, operator, lat, long, basin, formation, block
# cols_agree_needed = ['Country/Area', 'Subnational unit (province, state)', 'Status']
to_reconcile = {}
# loop through and see if values match, if they do not they get added to a tuple according to the project id
all_cols = cols_hier_top + cols_hier_middle + cols_agree_needed
# strip last character and de duplicate
all_cols = [col[:-1] for col in all_cols]
all_cols = list(set(all_cols))
all_groupings = all_groupings.fillna('')
for row in all_groupings.index:
    proj_id = all_groupings.loc[row, 'Unit IDp']
    proj_name = all_groupings.loc[row, 'Unit Namep']
    unit_name = all_groupings.loc[row, 'Unit Namef']
    unit_id = all_groupings.loc[row, 'Unit IDf']
    for col in all_cols:
        
        field_val = all_groupings.loc[row,f'{col}f']
        proj_val = all_groupings.loc[row,f'{col}p']
        if field_val != proj_val:
            if field_val == '' or proj_val == '':
                continue
            else:
                to_reconcile[(proj_id, proj_name, unit_id, unit_name, col)] = [field_val, proj_val]
                
to_reconcile_df = pd.DataFrame.from_dict(to_reconcile, orient='index')    
to_reconcile_df.to_excel(f'to_reconcile_goget_2025-03-11.xlsx', index=True)


In [90]:


# print(f'Ask Scott: {questions}')

